<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/ner-survey-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np

In [47]:
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/ArmanPersoNERCorpus/test.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/ArmanPersoNERCorpus/train.txt'

--2020-09-23 13:46:54--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/ArmanPersoNERCorpus/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815189 (2.7M) [text/plain]
Saving to: ‘test.txt’

test.txt            100%[===================>]   2.68M  12.6MB/s    in 0.2s    

2020-09-23 13:46:54 (12.6 MB/s) - ‘test.txt’ saved [2815189/2815189]

--2020-09-23 13:46:54--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/ArmanPersoNERCorpus/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [50]:
from pathlib import Path
import re

def read_arman(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split(' ')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [51]:
texts, tags = read_arman('train.txt')

In [53]:
!pip install transformers

     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 890kB 17.1MB/s 
     |████████████████████████████████| 3.0MB 26.6MB/s 
     |████████████████████████████████| 1.1MB 26.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=34ec9f082243dc68c15a2f18accc4ac8e6f679fd3d8f4fbfeb0f26eebc25badf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [57]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = TFAutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [63]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [66]:
train_encodings = tokenizer(texts, is_split_into_words=True, padding=True)

In [ ]:
# train_encodings = tokenizer(texts, is_split_into_words=True, padding=True)

In [67]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(tags, train_encodings)

AttributeError: ignored